In [146]:
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


### Linear Layer

In [294]:
class Linear:
    def __init__(self, input_size, output_size):
        
        self.weights=np.random.normal(0, 0.1,( input_size, output_size))
        self.bias=0
        self.in_size=input_size
        self.out_size=output_size

    def forward(self, input_x):
        self.input=input_x
        output=np.dot(input_x, self.weights) + self.bias
        return output

    def backward(self, grad_loss, lr):
        grad_W=np.dot(self.input.T, grad_loss)
        grad_Bias=np.sum(grad_loss)
        self.weights-=lr*grad_W/self.input.shape[0]
        self.bias-=lr*grad_Bias/ self.input.shape[0]
        return (np.dot(grad_loss,self.weights.T))

    def backwardL2reg(self, grad_loss, lr):
        grad_reg=np.dot(self.weight, grad_loss)
        grad_W=np.dot(self.input.T, grad_loss) + grad_reg
        grad_Bias=np.sum(grad_loss)
        self.weights-=lr*grad_W/self.input.shape[0]
        self.bias-=lr*grad_Bias/ self.input.shape[0]
        return (np.dot(grad_loss,self.weights.T))

    def backwardL1reg(self, grad_loss, lr):
        grad_reg=grad_loss
        grad_W=np.dot(self.input.T, grad_loss) + grad_reg
        grad_Bias=np.sum(grad_loss)
        self.weights-=lr*grad_W/self.input.shape[0]
        self.bias-=lr*grad_Bias/ self.input.shape[0]
        return (np.dot(grad_loss,self.weights.T))

### Convolutional Layer

In [295]:
class Conv2D:
    def __init__(self, in_channels,filter_size,filter_num, padding=0, stride=1, bias=True,  conv_layer=1):
        sigma=0.1
        self.filter_size=filter_size
        self.filter=filter_num
        self.padding=padding
        self.stride=stride
        self.in_channels=in_channels
        self.weights=np.random.normal(0, sigma, size=(filter_num,in_channels,filter_size, filter_size))
        # self.weights=np.random.randn(filter_num,in_channels,filter_size, filter_size)
        self.bias=np.zeros(self.filter)
        self.conv_layer=conv_layer
        
    def forward(self,input):
        batch_size, in_channels, height, width=input.shape
        #Input shape= (Batch_size, in_channels, in_height, in_width)
        pad_size=self.padding
        #adding padd to the input
        padded_input=np.zeros(shape=(batch_size, in_channels, height+2*pad_size, width+2*pad_size), dtype='int32')
        padded_input[:,:, pad_size:height+pad_size, pad_size: width+pad_size]=input
        # Reminder: Save Input for backward-prop
        self.input=padded_input
        # Simple Conv operation:
        self.out_height=((height-self.filter_size + 2*pad_size)//self.stride)+1
        self.out_width=((width-self.filter_size + 2*pad_size)//self.stride)+1
        self.out_channel=self.filter
        output=np.zeros(shape=(batch_size, self.out_channel,self.out_height, self.out_width ), dtype='float64')
        # Loop over every location in inp_height * inp_width for the whole batch
        offset=self.filter_size//2
        for depth in range(self.out_channel):
            for batch in range(batch_size):
                for m in range(offset,(height-offset), self.stride):
                    for n in range(offset,(width-offset), self.stride):
                        output[batch,depth,m-offset, n-offset]=\
                            np.sum(np.multiply(padded_input[batch,:, m-offset:m+offset+1,n-offset:n+offset+1],
                                               self.weights[depth]))
        # Output will be of the size (Batch_size, out_channels, out_height, out_width)
        return output
    
    def backward(self, grad_out, learning_rate):
        batch_size,channels, height, width = self.input.shape
        grad_weights = np.zeros(self.weights.shape)
        grad_bias = np.zeros(self.filter)
        flip_weights=np.flip(self.weights, axis=(2,3))
        grad_input=np.zeros(self.input.shape)
        for batch in range(batch_size):
            for depth in range(self.filter):
                for m in range(0,height - self.filter_size + 1, self.stride):
                    for n in range(0, width - self.filter_size + 1, self.stride):
                    # grad_W=grad_output* input[kernel_size]
                    # grad_W=
                        grad_weights[depth] += grad_out[batch, depth, m,n] * self.input[batch,:, m:m+self.filter_size, n:n+self.filter_size]
                        grad_bias[depth] += grad_out[batch, depth, m,n]
        
        if(self.conv_layer!=1):
            for batch in range(batch_size):
                for m in range(self.out_height):
                    for n in range(self.out_width):
                        vert_start = m * self.stride
                        vert_end = vert_start + self.filter_size
                        horiz_start = n * self.stride
                        horiz_end = horiz_start + self.filter_size
                        for depth in range(self.filter):
                            grad_input[batch, :, vert_start:vert_end, horiz_start:horiz_end]+= np.sum(grad_out[batch,depth, m,n] * flip_weights[depth], axis=0)
            
        
        self.weights -= learning_rate * grad_weights / batch
        self.bias -= learning_rate * grad_bias / batch

        return grad_input
        


### MaxPooling Layer

In [234]:
class MaxPool2D:
    def __init__(self, pool_size, stride,padding=0, dilation=1):
        self.pool_size=pool_size
        self.stride=stride
        self.padding=padding
    def forward(self, input_data):
        batch_size, in_channels, height, width=input_data.shape
        pad_size=self.padding
        padded_input=np.zeros(shape=(batch_size, in_channels, height+2*pad_size, width+2*pad_size), dtype='int32')
        padded_input[:,:, pad_size:height+pad_size, pad_size: width+pad_size]=input_data
        self.input=padded_input
        offset=self.pool_size//2
        out_height=(height//self.stride) 
        out_width=(width//self.stride) 

        output=np.zeros((batch_size, in_channels, out_height, out_width), dtype="float64")
        # print(output.shape)
        for m in range(0,(height-self.pool_size), self.stride):
            for n in range(0,(width-self.pool_size), self.stride):
                output[:,: ,m:m+1, n:n+1]= np.max(np.max(padded_input[:,:, m:m+self.pool_size,n:n+self.pool_size],axis=2, keepdims=True ), axis=3, keepdims=True)
                
        self.max_output=output
        return output
    
    def backward(self, grad_out, lr):
        batch_size, in_channels ,  height, width= self.input.shape
        grad_input = np.zeros_like(self.input)

        for batch in range(batch_size):
            for depth in range(in_channels):
                for m in range(0, height, self.pool_size):
                    for n in range(0, width, self.pool_size):
                        window = self.input[batch,depth, m:m+self.pool_size, n:n+self.pool_size]
                        max_val = np.max(window)
                        grad_input[batch,depth, m:m+self.pool_size, n:n+self.pool_size] = (window == max_val) * grad_out[batch,depth,  m//self.pool_size, n//self.pool_size]
        return grad_input
        

### Average pooling Layer

In [268]:
class AvgPool2D:
    def __init__(self, pool_size, stride,padding=0, dilation=1):
        self.pool_size=pool_size
        self.stride=stride
        self.padding=padding
    def forward(self, input_data):
        batch_size, in_channels, height, width=input_data.shape
        pad_size=self.padding
        padded_input=np.zeros(shape=(batch_size, in_channels, height+2*pad_size, width+2*pad_size), dtype='int32')
        padded_input[:,:, pad_size:height+pad_size, pad_size: width+pad_size]=input_data
        self.input=padded_input
        offset=self.pool_size//2
        out_height=(height//self.stride) 
        out_width=(width//self.stride) 

        output=np.zeros((batch_size, in_channels, out_height, out_width), dtype="float64")
        # print(output.shape)
        for m in range(0,(height-self.pool_size), self.stride):
            for n in range(0,(width-self.pool_size), self.stride):
                output[:,: ,m, n]=np.mean(padded_input[:,:, m:m+self.pool_size,n:n+self.pool_size])

    def backward(self, grad_loss, lr):
        return grad_loss/(self.pool_size)**2

### Flatten Layer

In [235]:
class Flatten:
    def __init__(self):
        self=None
    def forward(self, input_data):
        self.input=input_data
        batch=input_data.shape[0]
        return input_data.reshape(batch, -1)
    
    def backward(self, grad_out, lr):
        return grad_out.reshape(self.input.shape)

### Activation Layer

In [236]:
class activation:
  def __init__(self, func):
    self.func=func
  def forward(self, x):
      self.x=x
      if(self.func=='sigmoid'):
          loss= 1/(1+np.exp(-x))

      elif(self.func=='relu'):
          loss= np.maximum(0, x)
          
      elif(self.func=='softmax'):
          exp=np.exp(x)
          loss=exp/ np.sum(exp, axis=0)

      return loss
  def backward(self, grad_loss, lr):
      if(self.func=='sigmoid'):
          loss= 1/(1+np.exp(-self.x))
          back_loss= np.exp(-self.x)/ (loss**2)
      elif(self.func=='relu'):

          back_loss=1*(self.x>0)
      elif(self.func=='softmax'):
        
        t_exp = np.exp(self.x)
        # Sum of all e^totals
        S = np.sum(t_exp)

        back_loss = t_exp * (S - t_exp) / (S ** 2)
        
      
      return grad_loss*back_loss

###  Loss Functions

In [237]:
def oneHotVector(input_y, num_classes):
    batch=input_y.shape[0]
    one_hot_vector=np.zeros((batch, num_classes))
    for b in range(batch):
        one_hot_vector[b,int(input_y[b])]=1
    return one_hot_vector


In [288]:
def crossEntropy(predicted,input_y):
    row,col=input_y.shape
    regu=0
    epsilon = 1e-12
    predicted = np.clip(predicted, epsilon, 1. - epsilon)
    y1=np.log(predicted)
    
    return np.mean(-np.sum(input_y*y1)) , input_y/predicted

### Creating the Convolutional Neural Network Model

In [239]:
# compiling all the above layers in one CNN Model
class Model:
    def __init__(self):
        self.layers=[]
    def add(self, layer):
        self.layers.append(layer)
    def forward(self, input_data):
        output=input_data
        for layer in self.layers:
            output=layer.forward(output)
            print(output.shape)
        return output
    def backward(self,grad_out, lr):
        for layer in reversed(self.layers):
            grad_out=layer.backward(grad_out, lr)
            print(" gradout", grad_out.shape)
        

### Load KMNIST Data

In [156]:
train_data=np.load("kmnist-train-imgs.npz")["arr_0"]
test_data= np.load("kmnist-test-imgs.npz")['arr_0']
train_labels= np.load("kmnist-train-labels.npz")['arr_0']
test_labels=np.load("kmnist-test-labels.npz")['arr_0']

In [126]:
test_labels

array([2, 9, 3, ..., 9, 4, 2], dtype=uint8)

In [157]:
train_data=train_data.reshape(train_data.shape[0], 1, train_data.shape[1], train_data.shape[2])
test_data=test_data.reshape((test_data.shape[0], 1, test_data.shape[1], test_data.shape[2]))

In [128]:
train_data=train_data/255
train_data


array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]],


       [[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0

In [129]:
test_labels[3]

8

In [130]:
# Split train data in training and validation
train_X, validation_X=train_data[: 1000], train_data[1000:]
train_y, validation_y=train_labels[: 1000], train_labels[1000:]

In [131]:
train_y


array([8, 7, 0, 1, 4, 2, 4, 8, 1, 1, 5, 1, 0, 5, 7, 6, 1, 7, 9, 5, 7, 3,
       7, 5, 6, 6, 2, 7, 6, 0, 9, 6, 1, 5, 9, 5, 8, 0, 0, 8, 8, 6, 7, 7,
       7, 8, 1, 9, 6, 0, 5, 1, 1, 1, 3, 2, 2, 6, 4, 3, 5, 5, 4, 6, 6, 1,
       7, 8, 8, 3, 1, 9, 9, 3, 2, 1, 0, 4, 8, 2, 3, 6, 9, 9, 6, 5, 6, 1,
       0, 7, 2, 2, 8, 0, 1, 8, 6, 6, 3, 5, 0, 9, 6, 9, 8, 2, 1, 3, 9, 4,
       6, 5, 3, 4, 7, 0, 4, 8, 6, 5, 5, 4, 2, 2, 2, 5, 6, 6, 8, 1, 5, 5,
       6, 2, 9, 2, 0, 5, 5, 2, 7, 5, 6, 8, 8, 0, 3, 4, 5, 6, 3, 5, 0, 1,
       4, 4, 7, 7, 7, 8, 6, 6, 5, 2, 7, 5, 2, 1, 8, 1, 5, 5, 3, 6, 1, 0,
       1, 0, 3, 7, 0, 9, 9, 8, 0, 0, 0, 9, 5, 9, 6, 5, 4, 5, 3, 8, 8, 3,
       6, 0, 8, 4, 5, 7, 0, 8, 4, 5, 1, 1, 2, 3, 8, 7, 3, 7, 4, 7, 2, 9,
       1, 7, 9, 5, 7, 4, 6, 9, 0, 1, 0, 1, 2, 6, 5, 9, 0, 5, 7, 0, 6, 8,
       2, 1, 9, 7, 7, 9, 8, 6, 6, 1, 9, 1, 0, 8, 5, 2, 7, 1, 4, 5, 0, 5,
       4, 7, 1, 2, 3, 7, 0, 8, 5, 9, 5, 2, 3, 2, 3, 2, 7, 3, 4, 0, 9, 5,
       0, 3, 5, 4, 6, 3, 8, 6, 7, 4, 7, 1, 8, 4, 1,

In [132]:
output=np.ones((100, 64, 13,13), dtype="float64")

In [133]:
output[:,: , 0:1,0:1].shape

(100, 64, 1, 1)

In [134]:
binary_train_y=oneHotVector(train_y, 10)
binary_train_y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

### CNN Model Training

Model1

In [218]:
cnn_model1=Model()
cnn_model1.add(Conv2D(1, 5, 64))# 1000 x 64 x 24x 24
cnn_model1.add(MaxPool2D(2, 2)) # 1000 x64 x12x12
cnn_model1.add(activation('sigmoid'))
cnn_model1.add(Flatten())#1000x (64. 13.13)  = 1000x 1600
cnn_model1.add(Linear((64*12*12), 1000))
cnn_model1.add(Linear(1000, 10))
cnn_model1.add(activation('softmax'))


In [221]:
#Hyperparameters
epochs=10
lr=100000


In [220]:
train_loss=[]
accuracy=[]
for epoch in tqdm(range(epochs), total=epochs):
    total_samples=0
    total_correct=0
    output=cnn_model1.forward(train_X)
    # print(output)
    y=oneHotVector(train_y, 10)
    loss, grad_out= crossEntropy(output, y)
    cnn_model1.backward(grad_out, lr)
    predicted = np.argmax(output,axis=1)
    # print(predicted)
    total_samples += train_y.shape[0]
    total_correct += (predicted == train_y).sum().item()
    acc=total_correct/total_samples
    train_loss.append(loss)
    accuracy.append(acc)
    print("Epoch {}: loss= {}, accuracy= {}".format(epoch, loss/total_samples, acc*100))
    

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0: loss= 6.903957331009915, accuracy= 12.5
Epoch 1: loss= 6.903950705600238, accuracy= 12.5
Epoch 2: loss= 6.903943962828895, accuracy= 12.5
Epoch 3: loss= 6.903937283077984, accuracy= 12.5
Epoch 4: loss= 6.903930671391561, accuracy= 12.5
Epoch 5: loss= 6.90392412820671, accuracy= 12.5
Epoch 6: loss= 6.90391765389397, accuracy= 12.4
Epoch 7: loss= 6.903911248846949, accuracy= 12.4
Epoch 8: loss= 6.903904913484495, accuracy= 12.4
Epoch 9: loss= 6.903898648251147, accuracy= 12.4


Model2

In [266]:
cnn_model2=Model()
cnn_model2.add(Conv2D(1,5, 128, conv_layer=1))  # 1000 x 128 x 24x 24 
cnn_model2.add(MaxPool2D(2,2)) # 1000x 128 x 12 x12
cnn_model2.add(activation('sigmoid'))
cnn_model2.add(Conv2D(128, 3, 64, conv_layer=2))# 1000 x 64 x 10x 10
cnn_model2.add(MaxPool2D(2,2)) # 1000x 64x 5x5
cnn_model2.add(activation('sigmoid'))
cnn_model2.add(Flatten())#1000x (64. 5.5)  
cnn_model2.add(Linear((64*5*5), 100))
cnn_model2.add(Linear(100, 10))
cnn_model2.add(activation('softmax'))

In [267]:
epochs =12
lr=1000
train_loss=[]
accuracy=[]
for epoch in tqdm(range(epochs), total=epochs):
    total_samples=0
    total_correct=0
    output=cnn_model2.forward(train_X)
    
    y=oneHotVector(train_y, 10)
    loss, grad_out= crossEntropy(output, y)
    cnn_model2.backward(grad_out, lr)
    predicted = np.argmax(output,axis=1)
    total_samples += train_y.shape[0]
    total_correct += (predicted == train_y).sum().item()
    acc=total_correct/total_samples
    train_loss.append(loss)
    accuracy.append(acc)
    print("Epoch {}: loss= {}, accuracy= {}".format(epoch, loss/total_samples, acc*100))

  0%|          | 0/12 [00:00<?, ?it/s]

(1000, 128, 24, 24)
(1000, 128, 12, 12)
(1000, 128, 12, 12)
(1000, 64, 10, 10)
(1000, 64, 5, 5)
(1000, 64, 5, 5)
(1000, 1600)
(1000, 100)
(1000, 10)
(1000, 10)
 gradout (1000, 10)
 gradout (1000, 100)
 gradout (1000, 1600)
 gradout (1000, 64, 5, 5)
 gradout (1000, 64, 5, 5)
 gradout (1000, 64, 10, 10)
 gradout (1000, 128, 12, 12)
 gradout (1000, 128, 12, 12)
 gradout (1000, 128, 24, 24)
 gradout (1000, 1, 28, 28)
Epoch 0: loss= 6.907755278982138, accuracy= 10.100000000000001
(1000, 128, 24, 24)
(1000, 128, 12, 12)
(1000, 128, 12, 12)
(1000, 64, 10, 10)
(1000, 64, 5, 5)
(1000, 64, 5, 5)
(1000, 1600)
(1000, 100)
(1000, 10)
(1000, 10)
 gradout (1000, 10)
 gradout (1000, 100)
 gradout (1000, 1600)
 gradout (1000, 64, 5, 5)
 gradout (1000, 64, 5, 5)


/tmp/ipykernel_2590467/3492444217.py:13: RuntimeWarning: overflow encountered in exp
  exp=np.exp(x)
/tmp/ipykernel_2590467/3492444217.py:14: RuntimeWarning: invalid value encountered in divide
  loss=exp/ np.sum(exp, axis=0)
/tmp/ipykernel_2590467/3492444217.py:26: RuntimeWarning: overflow encountered in exp
  t_exp = np.exp(self.x)
/tmp/ipykernel_2590467/3492444217.py:30: RuntimeWarning: invalid value encountered in divide
  back_loss = -t_exp * t_exp / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:31: RuntimeWarning: invalid value encountered in subtract
  back_loss = t_exp * (S - t_exp) / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:31: RuntimeWarning: invalid value encountered in multiply
  back_loss = t_exp * (S - t_exp) / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:31: RuntimeWarning: invalid value encountered in divide
  back_loss = t_exp * (S - t_exp) / (S ** 2)


 gradout (1000, 64, 10, 10)
 gradout (1000, 128, 12, 12)
 gradout (1000, 128, 12, 12)
 gradout (1000, 128, 24, 24)
 gradout (1000, 1, 28, 28)
Epoch 1: loss= nan, accuracy= 8.7
(1000, 128, 24, 24)
(1000, 128, 12, 12)
(1000, 128, 12, 12)


/tmp/ipykernel_2590467/3492444217.py:7: RuntimeWarning: overflow encountered in exp
  loss= 1/(1+np.exp(-x))


...

/tmp/ipykernel_2590467/3492444217.py:19: RuntimeWarning: overflow encountered in exp
  loss= 1/(1+np.exp(-self.x))
/tmp/ipykernel_2590467/3492444217.py:20: RuntimeWarning: overflow encountered in exp
  back_loss= np.exp(-self.x)/ (loss**2)


 gradout (1000, 128, 12, 12)


/tmp/ipykernel_2590467/4239141351.py:35: RuntimeWarning: invalid value encountered in multiply
  grad_input[batch,depth, m:m+self.pool_size, n:n+self.pool_size] = (window == max_val) * grad_out[batch,depth,  m//self.pool_size, n//self.pool_size]


 gradout (1000, 128, 24, 24)
 gradout (1000, 1, 28, 28)
Epoch 2: loss= nan, accuracy= 8.7
(1000, 128, 24, 24)
(1000, 128, 12, 12)
(1000, 128, 12, 12)
(1000, 64, 10, 10)
(1000, 64, 5, 5)
(1000, 64, 5, 5)
(1000, 1600)
(1000, 100)
(1000, 10)
(1000, 10)
 gradout (1000, 10)
 gradout (1000, 100)
 gradout (1000, 1600)
 gradout (1000, 64, 5, 5)
 gradout (1000, 64, 5, 5)
 gradout (1000, 64, 10, 10)
 gradout (1000, 128, 12, 12)
 gradout (1000, 128, 12, 12)
 gradout (1000, 128, 24, 24)
 gradout (1000, 1, 28, 28)
Epoch 3: loss= nan, accuracy= 8.7


KeyboardInterrupt: 

In [249]:
import random
binary_data=np.loadtxt("/data/home1/shreeyagarg/AIP/Assignment4/binary_classification_data_group_6_test.txt", delimiter="\t", skiprows=1)
idx=random.sample(range(binary_data.shape[0]), 1000)
binarydata=binary_data[:, :10]
labels=binary_data[:, 10]
training_data=binarydata[idx]
training_labels=labels[idx]
res=[ele for i, ele in enumerate(list(range(binary_data.shape[0]))) if i not in idx]
testing_data=binary_data[res]
testing_labels=labels[res]

In [252]:
training_data=training_data.reshape(training_data.shape[0], 1, 10, 1)

In [292]:
cnn_model3=Model()
cnn_model3.add(Conv2D(1,1, 128, conv_layer=1))  # 1000 x 128 x 10x1 
# cnn_model3.add(activation('relu'))
cnn_model3.add(Conv2D(128, 1, 64, conv_layer=2))# 1000 x 64 x 10x 1
# cnn_model3.add(activation('relu'))
cnn_model3.add(Flatten())#1000x (64. 10.1)  
cnn_model3.add(Linear((640), 2))
# cnn_model3.add(Linear(100, 2))
cnn_model3.add(activation('softmax'))

In [293]:
epochs =20
lr=0.001
train_loss=[]
accuracy=[]
for epoch in tqdm(range(epochs), total=epochs):
    total_samples=0
    total_correct=0
    output=cnn_model3.forward(training_data)
    
    y=oneHotVector(training_labels, 2)
    loss, grad_out= crossEntropy(output, y)
    cnn_model3.backward(grad_out, lr)
    predicted = np.argmax(output,axis=1)
    total_samples += train_y.shape[0]
    total_correct += (predicted == train_y).sum().item()
    acc=total_correct/total_samples
    train_loss.append(loss)
    accuracy.append(acc)
    print("Epoch {}: loss= {}, accuracy= {}".format(epoch, loss/total_samples, acc*100))

  0%|          | 0/20 [00:00<?, ?it/s]

(1000, 128, 10, 1)
(1000, 64, 10, 1)
(1000, 640)
(1000, 2)
(1000, 2)
 gradout (1000, 2)
 gradout (1000, 640)
 gradout (1000, 64, 10, 1)


/tmp/ipykernel_2590467/3492444217.py:30: RuntimeWarning: overflow encountered in multiply
  back_loss = -t_exp * t_exp / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:30: RuntimeWarning: overflow encountered in double_scalars
  back_loss = -t_exp * t_exp / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:30: RuntimeWarning: invalid value encountered in divide
  back_loss = -t_exp * t_exp / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:31: RuntimeWarning: overflow encountered in multiply
  back_loss = t_exp * (S - t_exp) / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:31: RuntimeWarning: overflow encountered in double_scalars
  back_loss = t_exp * (S - t_exp) / (S ** 2)
/tmp/ipykernel_2590467/3492444217.py:31: RuntimeWarning: invalid value encountered in divide
  back_loss = t_exp * (S - t_exp) / (S ** 2)


 gradout (1000, 128, 10, 1)
 gradout (1000, 1, 10, 1)
Epoch 0: loss= 27.603390094812656, accuracy= 8.5
(1000, 128, 10, 1)
(1000, 64, 10, 1)
(1000, 640)
(1000, 2)
(1000, 2)
 gradout (1000, 2)
 gradout (1000, 640)
 gradout (1000, 64, 10, 1)
 gradout (1000, 128, 10, 1)
 gradout (1000, 1, 10, 1)
Epoch 1: loss= nan, accuracy= 8.7


KeyboardInterrupt: 